# 🔄 Camada Prata - Dados Transformados e Relacionados

Este notebook demonstra como visualizar e consultar dados da camada Prata usando Spark.

## O que é a Camada Prata?
- Dados limpos e normalizados da camada Bronze
- Relacionamentos entre tabelas estabelecidos
- Estruturas dimensionais e de fatos criadas
- Prontos para análises mais complexas

In [ ]:
# ⚠️ IMPORTANTE: Execute primeiro o notebook CONFIGURAR_SPARK.ipynb
# Se ainda não executou, execute o script de correção:
try:
    spark
    print("✅ Spark Session já disponível")
except NameError:
    print("⚠️  Spark não configurado. Execute CONFIGURAR_SPARK.ipynb primeiro!")
    print("Ou execute o script de correção:")
    exec(open('/home/jovyan/work/fix_spark_py4j.py').read())

## 1. Listar Tabelas Disponíveis na Camada Prata

In [ ]:
# Listar arquivos Prata disponíveis
prata_paths = [
    "s3a://govbr/prata/dim_municipio/",
    "s3a://govbr/prata/dim_estado/",
    "s3a://govbr/prata/dim_orgao/",
    "s3a://govbr/prata/fact_bpc/",
    "s3a://govbr/prata/fact_populacao/"
]

print("📁 Tabelas disponíveis na Camada Prata:\n")
for path in prata_paths:
    try:
        df = spark.read.parquet(path)
        print(f"✅ {path.split('/')[-2]}")
        print(f"   Registros: {df.count():,}")
        print(f"   Colunas: {len(df.columns)}")
        print()
    except Exception as e:
        print(f"⚠️  {path.split('/')[-2]} - Não disponível")
        print()

## 2. Dimensão: Municípios

In [ ]:
# Ler dimensão de municípios
df_dim_municipio = spark.read.parquet("s3a://govbr/prata/dim_municipio/")

print(f"📊 Total de municípios: {df_dim_municipio.count():,}")
print(f"\n📋 Estrutura:")
df_dim_municipio.printSchema()

print(f"\n🔍 Exemplo de registros:")
df_dim_municipio.show(10, truncate=False)

## 3. Dimensão: Estados

In [ ]:
# Ler dimensão de estados
df_dim_estado = spark.read.parquet("s3a://govbr/prata/dim_estado/")

print(f"📊 Total de estados: {df_dim_estado.count()}")
print(f"\n🔍 Todos os estados:")
df_dim_estado.select("estado_id", "uf_sigla", "uf_nome", "regiao_nome").show(truncate=False)

## 4. Fato: BPC (Benefício de Prestação Continuada)

In [ ]:
# Ler fato BPC
try:
    df_fact_bpc = spark.read.parquet("s3a://govbr/prata/fact_bpc/")
    
    print(f"📊 Total de registros BPC: {df_fact_bpc.count():,}")
    print(f"\n📋 Estrutura:")
    df_fact_bpc.printSchema()
    
    print(f"\n🔍 Primeiros registros:")
    df_fact_bpc.show(10, truncate=False)
    
except Exception as e:
    print(f"⚠️  Dados não disponíveis: {e}")

## 5. Análises com Relacionamentos

In [ ]:
# Criar views temporárias
df_dim_municipio.createOrReplaceTempView("dim_municipio")
df_dim_estado.createOrReplaceTempView("dim_estado")

if 'df_fact_bpc' in locals():
    df_fact_bpc.createOrReplaceTempView("fact_bpc")
    
    # Análise: BPC por estado com informações da dimensão
    resultado = spark.sql("""
        SELECT 
            e.uf_sigla,
            e.uf_nome,
            e.regiao_nome,
            COUNT(DISTINCT f.municipio_id) as municipios_com_bpc,
            SUM(f.valor) as valor_total,
            SUM(f.quantidade_beneficiados) as total_beneficiados
        FROM fact_bpc f
        JOIN dim_municipio m ON f.municipio_id = m.municipio_id
        JOIN dim_estado e ON m.estado_id = e.estado_id
        GROUP BY e.uf_sigla, e.uf_nome, e.regiao_nome
        ORDER BY valor_total DESC
    """)
    
    print("📊 BPC por Estado (com relacionamentos):")
    resultado.show(truncate=False)
else:
    print("⚠️  Dados de BPC não disponíveis para análise")

## 6. Análise de Municípios por Região

In [ ]:
# Análise usando SQL
resultado = spark.sql("""
    SELECT 
        e.regiao_nome,
        COUNT(DISTINCT m.municipio_id) as total_municipios,
        COUNT(DISTINCT e.estado_id) as total_estados
    FROM dim_municipio m
    JOIN dim_estado e ON m.estado_id = e.estado_id
    GROUP BY e.regiao_nome
    ORDER BY total_municipios DESC
""")

print("📊 Municípios por Região:")
resultado.show(truncate=False)

## ✅ Resumo

A camada Prata contém dados transformados e relacionados:
- ✅ Dimensões: Municípios, Estados, Órgãos
- ✅ Fatos: BPC, População
- ✅ Relacionamentos estabelecidos entre tabelas
- ✅ Pronto para análises mais complexas